In [ ]:
# With due credits to Crew AI (https://www.crewai.com 
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain-groq

In [2]:
import warnings 
warnings.filterwarnings("ignore")

In [12]:
from crewai import Agent, Task , Crew

In [13]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key() 
# Please restart kernel to load new env variables
MODEL = ['gpt-4o', 'gpt-4' , 'gpt-3.5-turbo'] # to change models
os.environ["OPENAI_MODEL_NAME"] = MODEL[2]

In [ ]:
# IF WANT TO USE GROQ 
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     openai_api_base="https://api.groq.com/openai/v1",
#     openai_api_key=os.environ['GROQ_API_KEY'],
#     model_name="llama3-8b-8192",
#     temperature=0,
#     max_tokens=1000,
# )

### Creating Agents

In [14]:
Planner = Agent(
    role = "Plan the Route to be taken",
    goal = "Understand the user {topic}and decide whether to converse more with user or perform a vector database search",
    backstory = "You are an expert at routing user query: {topic}."
              "If the user asks for educational information, you do a vector database search and retrieve the relevant answer"
              "However if the user wants to help thier family member OR a friend, you engage in further converation and provide emphathetic resposne",
    allow_delegation = False,
    verbose = True
    # llm = llm
)

In [ ]:
# from crewai_tools  import tool
# @tool
# def router_tool(query):
#   if 'help' in query:
#     return 'plan'
#   else:
#     return 'retrieve'

In [15]:
Retriever = Agent(
        role="Retriever",
        goal="Use the information retrieved from the vectorstore to answer the {topic}",
        backstory=(
            "You are an assistant for question-answering tasks."
            "Use the information present in the retrieved context to answer the question."
            "You have to provide a clear concise answer."
        ),
        verbose=True,
        allow_delegation=False # attach llamindex rag here
        # llm=llm,
)

### Creating Tasks

In [16]:
plan = Task(
    description=(
        "1. Analyze the user's {topic} to determine the nature of the request.\n"
        "2. If the query is educational, perform a vector database search to retrieve the most relevant and accurate information.\n"
        "3. If the query is related to helping a family member or friend, engage in a conversation to provide a supportive and empathetic response.\n"
        "4. Ensure the response is detailed and accurately addresses the user's needs."
    ),
    expected_output="A decision on whether to perform a vector database search or engage in a conversation based on the user's query, "
                    "with a detailed and appropriate response provided accordingly.",
    agent=Planner,
)

In [17]:
retrieve = Task(
    description=(
        "1. Analyze the question to understand the user's {topic}.\n"
        "2. Perform a search in the vectorstore to retrieve the most relevant context.\n"
        "3. Use the retrieved context to formulate a clear and concise answer.\n"
        "4. Ensure the answer is accurate and directly addresses the question."
    ),
    expected_output="A clear and concise answer to the user's question, based on the information retrieved from the vectorstore.",
    agent=Retriever,
)

### starting the crew

In [18]:
crew = Crew(
    agents=[Planner, Retriever],
    tasks=[plan, retrieve], # order matter
    verbose=2
)

2024-06-26 18:42:51,855 - 4794308096 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [19]:
TOPIC = "I want to talk about my son who's having stress and low confidence for some time"
result = crew.kickoff(inputs={"topic": TOPIC})

 [DEBUG]: == Working Agent: Plan the Route to be taken
 [INFO]: == Starting Task: 1. Analyze the user's I want to talk about my son who's having stress and low confidence for some time to determine the nature of the request.
2. If the query is educational, perform a vector database search to retrieve the most relevant and accurate information.
3. If the query is related to helping a family member or friend, engage in a conversation to provide a supportive and empathetic response.
4. Ensure the response is detailed and accurately addresses the user's needs.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 
Based on your statement, "I want to talk about my son who's having stress and low confidence for some time," it seems like you are seeking advice or guidance on how to help your son navigate through his current challenges. As this query is related to helping a family member, I will engage in a conversation to provide a supportive and empathetic 

The PDFSearchTool is a RAG tool designed for semantic searches within PDF content. It allows for inputting a search query and a PDF document, leveraging advanced search techniques to find relevant content efficiently. This capability makes it especially useful for extracting specific information from large PDF files quickly.

In [ ]:
# from crewai_tools import PDFSearchTool

# rag_tool = PDFSearchTool(pdf='/content/17.pdf',
#     config=dict(
#         llm=dict(
#             provider="groq", # or google, openai, anthropic, llama2, ...
#             config=dict(
#                 model="llama3-8b-8192",
#                 # temperature=0.5,
#                 # top_p=1,
#                 # stream=true,
#             ),
#         ),
#         embedder=dict(
#             provider="huggingface", # or openai, ollama, ...
#             config=dict(
#                 model="BAAI/bge-small-en-v1.5",
#                 #task_type="retrieval_document",
#                 # title="Embeddings",
#             ),
#         ),
#     )
# )